In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

rc = {"font.family" : "serif", 
      "mathtext.fontset" : "stix"}
plt.rcParams.update(rc)
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]
matplotlib.rcParams.update({'font.size': 14})

In [2]:
def foo_k(T, gas_name):
    if gas_name == 'H2':
        a0 = 0.3975
        a1 = 0.4814/10**4
        a2 = -1.073/10**7
        a3 = 4.6863/10**11
        a4 = -8.5361/10**15
        a5 = 5.6934/10**19
    elif gas_name == 'O2':
        a0 = 0.4697
        a1 = -2.9819/10**4
        a2 = 2.0320/10**7
        a3 = -7.1938/10**11
        a4 = 12.311/10**15
        a5 = -8.0736/10**19
    elif gas_name == 'H2O':
        a0 = 0.3834
        a1 = -1.8579/10**4
        a2 = 0.6266/10**7
        a3 = -1.0528/10**11
        a4 = 0.93554/10**15
        a5 = -0.38512/10**19
    else:
        print('Houston we have a problem')
    return 1 + a0 + a1*T + a2*T**2 + a3*T**3 + a4*T**4 + a5*T**5

In [10]:
def foo_alpha(r):
    return (1-(1-math.e**(-r))/r)/(1-math.e**(-r))

In [14]:
foo_alpha(1000)

0.999

In [15]:
foo_alpha(0)

ZeroDivisionError: float division by zero

In [13]:
foo_alpha(-1000)

OverflowError: (34, 'Result too large')

In [4]:
# параметры установки
d = 45*10**(-3) # калибр
S = math.pi*d**2/4 # площадь
W_0 = 0.005 # объем каморы
l_d = 4.5 # длина ствола
m_elem = 0.52 # масса МЭ
p_0 = 34.5 * 10**6 # начальное давление

Q_t = 241.8 * 10**3 # энергия образования 1 кг продуктов реакций ???
E_ign = 60*10**3 # энергия активации

v_p0 = 0 # начальная скорость
x_p0 = 0 # начальная координата
p_a = 10**5 # атмосфэрное давление

In [5]:
# термодинамика
mu_H2 = 2*10**(-3) # молярная масса водорода
mu_O2 = 32*10**(-3) # молярная масса кислорода
mu_H2O = 18*10**(-3) # молярная масса воды

p_0 = 34.5 * 10**6 # начальное давления
T_0 = 300 # начальная температура
R = 8.31446262 # универсальная газовая постоянная
rho_0 = p_0/(R*T_0) # начальная плотность газа
m_g_0 = rho_0*W_0 # начальная масса газа

hi_O2_and_H2_0 = 8 # начальное соотношение мольных долей
ratio_m_O2_and_m_H2_0 = hi_O2_and_H2_0*mu_O2/mu_H2 # начальное соотнеошение масс газа через мольные доли, скрин "мольные доли.png"

m_H2_0 = m_g_0 / (ratio_m_O2_and_m_H2_0+1) # начальная масса водорода
m_O2_0 = m_H2_0 * ratio_m_O2_and_m_H2_0 # начальная масса кислорода

n_H2_0 = m_H2_0/mu_H2 # начальное количества вещества водорода
n_O2_0 =  m_O2_0/mu_O2 # начальное количества вещества кислорода

С_H2_0 = n_H2_0/W_0 # начальная молярная концентрация водорода
С_O2_0 = n_O2_0/W_0 # начальная молярная концентрация кислорода

C_H2O_0 = 0 # нет продуктов реакции в начальный момент времени

In [9]:
tau = 0 # время от начала процесса
C_n_1 = np.array([С_H2_0, С_O2_0, C_H2O_0]) # начальный вектор C
P_n = np.array([0, 0, foo_k(T_0,'H2O')*C_n_1[0]*C_n_1[1]**(1/2)]) # начальный вектор Р
D_n = np.array([foo_k(T_0,'H2')*C_n_1[1]/2, 1/2*foo_k(T_0, 'O2')*C_n_1[0]*C_n_1[1]**(1/2),0]) # начальный вектор D
W_n = W_0 # начальный объем
v_p = v_p0 # начальная скорость

tau_arr = []
C_H2_arr = []
C_O2_arr = []
C_H2O_arr = []
v_p_arr = []
p_m_arr = []
x_p_arr = []
M_p_arr = []
rho_arr = []
T_arr = []

tau_arr = np.array(tau_arr)
C_H2_arr = np.array(C_H2_arr)
C_O2_arr = np.array(C_O2_arr)
C_H2O_arr = np.array(C_H2O_arr)
v_p_arr = np.array(v_p_arr)
p_m_arr = np.array(p_m_arr)
x_p_arr = np.array(x_p_arr)
M_p_arr = np.array(M_p_arr)
rho_arr = np.array(rho_arr)
T_arr = np.array(T_arr)

dt=1e-5
x_p = x_p0
T = T_0

k=0

while x_p <= l_d: # условие цикла, пока МЭ не покинул дульный срез
    k+=1
    tau += dt # время от начала процесса
    tau_arr = np.append(tau_arr, tau)
    
    C_n = C_n_1 # вектор С
    C_H2_arr = np.append(C_H2_arr, C_n[0])
    C_O2_arr = np.append(C_O2_arr, C_n[1])
    C_H2O_arr = np.append(C_H2O_arr, C_n[2])
    
    n_H2 = C_n[0] * W_n # количество вещества водорода
    n_O2 = C_n[1] * W_n # количество вещества кислорода
    n_H2O = C_n[2] * W_n # количество вещества воды
    
    m_H2 = n_H2 * mu_H2 # масса водорода
    m_O2 = n_O2 * mu_O2 # масса кислорода
    m_H2O = n_H2O * mu_H2O # масса воды
    m_g = m_H2 + m_O2 + m_H2O # масса смеси
    
    rho = m_g/W_n #плотность смеси
    rho_arr = np.append(rho_arr,rho)
    
    print(C_n)
    P_n = np.array([0, 0, foo_k(T,'H2O')*C_n[0]*C_n[1]**(1/2)]) # вектор P
    D_n = np.array([foo_k(T,'H2')*C_n[1]/2, 1/2*foo_k(T, 'O2')*C_n[0]*C_n[1]**(1/2),0]) # вектор D
    

    M_p = mu_H2O * C_n[2] * W_n # масса продуктов реакции
    M_p_arr = np.append(M_p_arr, M_p)

    phi = (1+3*m_g/m_elem) # фиктивность, масса всех продуктов?
    
    p_m = (foo_k(T,'H2O')-1)/W_n * ((p_0*W_0)/(foo_k(T, 'H2O')-1) + Q_t*M_p - E_ign - phi/2*m_elem*v_p**2) # среднебаллистическое давление
    p_m_arr = np.append(p_m_arr, p_m)
    
    T = p_m/(R*rho)
    T_arr = np.append(T_arr,T)
    
    dv_p = (p_m - p_a)*S/(phi*m_elem) * dt # диф. скорости МЭ
    
    v_p += dv_p # скорость МЭ
    v_p_arr = np.append(v_p_arr, v_p)
    
    dx_p = dv_p * dt # диф. координаты МЭ
    
    x_p += dx_p  # координата МЭ
    x_p_arr = np.append(x_p_arr, x_p)
    
    W_n_1 = W_0 + x_p*S  # Заснарядный объем на n+1 шаге
    
    W_n = W_0 + (x_p-dx_p)*S # Заснарядный объем на n шаге
           
    foo_alpha_res = np.array([]) # значение умножения алфа функции от произведения вектора D на tau, 
    for i in range(len(D_n)):
        foo_alpha_res = np.append(foo_alpha_res, foo_alpha(D_n[i]*tau))
        
    C_n_kr = W_n/W_n_1 * (C_n + (tau *(P_n-C_n*D_n))/(1+foo_alpha_res*D_n*tau)) # Предиктор вектора С

    D_n_kr = (D_n*C_n_kr + D_n*C_n)/2 # Предиктор вектора D
    
    P_n_kr = foo_alpha_res*P_n*C_n_kr + (1-foo_alpha_res)*P_n*C_n # Предиктор вектора P

    C_n_1 = W_n/W_n_1 * (C_n + (tau *(P_n_kr-C_n*D_n_kr))/(1+foo_alpha_res*D_n_kr*tau)) # Корректор вектора С

[ 53609.77064667 428878.16517338      0.        ]
[ 5.73546712e+03 -4.26898766e+05  1.75203938e+07]
[ 5.73546712e+03 -4.26898766e+05  1.75203938e+07]
[8115.51240126           nan           nan]
[8115.51240126           nan           nan]
[nan nan nan]


C:\Users\Stepan\AppData\Local\Temp\ipykernel_6928\3645793755.py:59: RuntimeWarning: invalid value encountered in double_scalars
  P_n = np.array([0, 0, foo_k(T,'H2O')*C_n[0]*C_n[1]**(1/2)]) # вектор P
C:\Users\Stepan\AppData\Local\Temp\ipykernel_6928\3645793755.py:60: RuntimeWarning: invalid value encountered in double_scalars
  D_n = np.array([foo_k(T,'H2')*C_n[1]/2, 1/2*foo_k(T, 'O2')*C_n[0]*C_n[1]**(1/2),0]) # вектор D
